In [ ]:
import numpy as np
import random
import math
import matplotlib.pyplot as plt
from benchmarkfcns import *
import pandas as pd

# Benchmark functions

## unimodal

### 2D

In [ ]:
unimodal_functions2D = {
    "AckleyN2": [ackleyn2, [-32, 32]],
    "BohachevskyN1": [bohachevskyn1, [-100, 100]],
    "Booth": [booth, [-10, 10]],
    "Brent": [brent, [-20, 0]], 
    "dropwave": [dropwave, [-5.2, 5.2]], 
    "Exponential": [exponential, [-1, 1]], 
    "Leon": [leon, [0, 10]],
    "Matyas": [matyas, [-10, 10]],
    "SchafferN1": [schaffern1, [-100, 100]],
    "SchafferN2": [schaffern2, [-100, 100]],
    "SchafferN3": [schaffern3, [-100, 100]],
    "SchafferN4": [schaffern4, [-100, 100]],
    "ThreeHumpCamel": [threehumpcamel, [-5, 5]]
}

### nD

In [ ]:
unimodal_functions30D = {
    "Brown": [brown, [-1, 4]],
    "Griewank": [griewank, [-600, 600]],
    "PowellSum": [powellsum, [-1, 1]],
    "Ridge": [ridge, [-5, 5]],
    "Schwefel2_20": [schwefel220, [-100, 100]],
    "Schwefel2_21": [schwefel221, [-100, 100]],
    "Schwefel2_22": [schwefel222, [-100, 100]],
    "Schwefel2_23": [schwefel223, [-10, 10]],
    "Sphere": [sphere, [-5.2, 5.2]],
    "SumSquares": [sumsquares, [-10, 10]],
    "Trid": [trid, [-10, 10]],
    "XinSheYangN3": [xinsheyangn3, [(-2)*np.pi, 2*np.pi]],
    "Zakharov": [zakharov, [-5, 10]]
}

## multimodal

### 1D

In [ ]:
multimodal_1d = {
    "Forrester": [forrester, [0, 1]],
    "Gramacy & Lee": [gramacylee, [0.5, 2.5]]
}

### 2D

In [ ]:
multimodal_2d = {
    "Adjiman": [adjiman, [-1, 2]],  # x1 in [-1,2], x2 in [-1,1] — simplified
    "Bartels Conn": [bartelsconn, [-500, 500]],
    "Beale": [beale, [-4.5, 4.5]],
    "Bird": [bird, [-2*math.pi, 2*math.pi]],
    "Bohachevsky N. 2": [bohachevskyn2, [-100, 100]],
    "Bukin N. 6": [bukinn6, [-15, -5]],  # x1 range, x2 in [-3,3]
    "Carrom Table": [carromtable, [-10, 10]],
    "Cross-in-Tray": [crossintray, [-10, 10]],
    "Deckkers-Aarts": [deckkersaarts, [-20, 20]],
    "Easom": [easom, [-100, 100]],
    "Egg Crate": [eggcrate, [-5, 5]],
    "El-Attar-Vidyasagar-Dutta": [elattar, [-500, 500]],
    "Goldstein-Price": [goldsteinprice, [-2, 2]],
    "Himmelblau": [himmelblau, [-5, 5]],
    "Holder-Table": [holdertable, [-10, 10]],
    "Keane": [keane, [0, 10]],
    "Levi N. 13": [levin13, [-10, 10]],
    "McCormick": [mccormick, [-1.5, 4]],  # x1 range, x2 in [-3,4]
    "Shubert 3": [shubertn3, [-10, 10]],
    "Shubert N. 4": [shubertn4, [-10, 10]],
    "Shubert": [shubert, [-10, 10]]                 
}

### 3D

In [ ]:
multimodal_3d = {
    "Wolfe": [wolfe, [0, 2]]        
}

### nD

In [ ]:
multimodal_30d = {
    "Alpine N. 1": [alpinen1, [0, 10]],
    "Alpine N. 2": [alpinen2, [0, 10]],
    "Happy Cat": [happycat, [-2, 2]],
    "Periodic": [periodic, [-10, 10]],
    "Qing": [qing, [-500, 500]],
    "Quartic": [quartic, [-1.28, 1.28]],
    "Rastrigin": [rastrigin, [-5.12, 5.12]],
    "Rosenbrock": [rosenbrock, [-5, 10]],
    "Salomon": [salomon, [-100, 100]],
    "Schwefel": [schwefel, [-500, 500]],
    "Styblinski-Tank": [styblinskitank, [-5, 5]],
    "Xin-She Yang": [xinsheyangn2, [-5, 5]],
    "Xin-She Yang N. 2": [xinsheyangn2, [-2*math.pi, 2*math.pi]],
    "Xin-She Yang N. 4": [xinsheyangn4, [-10, 10]],
    "Ackley": [ackley, [-32.768, 32.768]],
    "Ackley N. 4": [ackleyn4, [-32, 32]]  
}

# GA

In [ ]:
class Chromosome:
    def __init__(self, values):
        self.values = np.array(values)
        self.fitness = None

    def __repr__(self):
        return f"({self.values})"

def initialize_population(pop_size, bounds, dimension):
    population = []
    for _ in range(pop_size):
        values = np.random.uniform(bounds[0], bounds[1], dimension)
        population.append(Chromosome(values))
    return population

def evaluate_population(population, func):
    for chrom in population:
        chrom.fitness = func([chrom.values])

def selection(population, tournament_size=3):
    selected = random.sample(population, tournament_size)
    return min(selected, key=lambda chrom: chrom.fitness)

def crossover(parent1, parent2, crossover_rate, dimension):
    if random.random() < crossover_rate:
        # Blend crossover for real numbers
        alpha = random.random()
        child_values = alpha * parent1.values + (1 - alpha) * parent2.values
        return Chromosome(child_values)
    else:
        return random.choice([parent1, parent2])

def mutation(chrom, mutation_rate, bounds, dimension):
    if random.random() < mutation_rate:
        # Add small random value to each dimension
        mutation_values = np.random.uniform(-0.3, 0.3, dimension) * (bounds[1] - bounds[0])
        chrom.values += mutation_values
        # Ensure within bounds
        chrom.values = np.clip(chrom.values, bounds[0], bounds[1])
    return chrom

def create_new_generation(previous_gen, crossover_rate, mutation_rate, bounds, func, dimension):
    new_gen = []
    # Elitism: keep the best chromosome
    best_chrom = min(previous_gen, key=lambda chrom: chrom.fitness)
    new_gen.append(Chromosome(best_chrom.values.copy()))
    
    while len(new_gen) < len(previous_gen):
        parent1 = selection(previous_gen)
        parent2 = selection(previous_gen)
        child = crossover(parent1, parent2, crossover_rate, dimension)
        child = mutation(child, mutation_rate, bounds, dimension)
        new_gen.append(child)
    
    return new_gen

def genetic_algorithm(func, bounds, dimension=30, pop_size=50, crossover_rate=0.75, mutation_rate=0.01, max_generation=40000, patience=1000):
    population = initialize_population(pop_size, bounds, dimension)
    evaluate_population(population, func)

    best_chrom = min(population, key=lambda chrom: chrom.fitness)
    best_fitness = best_chrom.fitness
    best_fitness_per_generation = [best_fitness]

    no_improve_count = 0

    for gen in range(1, max_generation + 1):
        population = create_new_generation(population, crossover_rate, mutation_rate, bounds, func, dimension)
        evaluate_population(population, func)

        current_best = min(population, key=lambda chrom: chrom.fitness)

        if current_best.fitness < best_fitness:
            best_fitness = current_best.fitness
            best_chrom = current_best
            no_improve_count = 0
        else:
            no_improve_count += 1

        best_fitness_per_generation.append(best_fitness)

        if no_improve_count >= patience:
            print(f"Early stopping at generation {gen} (no improvement in {patience} generations)")
            break

    return best_chrom, best_fitness_per_generation

# GA results

## uni

### 2D

In [ ]:
dimension = 2
pop_size = 50
crossover_rate = 0.75
mutation_rate = 0.01
max_generation = 40000
patience = 10000

results = {}
for name, [func, bounds] in unimodal_functions2D.items():
    print(f"Running GA on {name}...")
    best_sol, fitness_history = genetic_algorithm(
        func, bounds, dimension, pop_size, crossover_rate, mutation_rate, max_generation, patience
    )
    results[name] = {
        'best_solution': best_sol.values,
        'best_fitness': best_sol.fitness,
        'fitness_history': fitness_history
    }
    print(f"Best solution for {name}: {best_sol.values} with fitness {best_sol.fitness}")

# Plot fitness history for each function
plt.figure(figsize=(12, 8))
for name, data in results.items():
    plt.plot(data['fitness_history'], label=name)

plt.xlabel('Generation')
plt.ylabel('Best Fitness')
plt.title('Genetic Algorithm Performance on Benchmark Functions (30-dimensional)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
print("\nSummary Results:")
print("=" * 60)
for name, data in results.items():
    print(f"{name:20}: Best Fitness = {data['best_fitness'][0]:.6e}")

#### statistics

In [ ]:
results = {}
all_func_names = []
all_fits_avg = []
all_fits_median = []
all_fits_std = []


for name, [func, bounds] in unimodal_functions2D.items():
    best_fits = []
    for _ in range(20):
            
        print(f"Running GA on {name}...")
        best_sol, fitness_history = genetic_algorithm(
            func, bounds, dimension, pop_size, crossover_rate, mutation_rate, max_generation, patience
        )
        results[name] = {
            'best_solution': best_sol.values,
            'best_fitness': best_sol.fitness,
            'fitness_history': fitness_history
        }
        print(f"Best solution for {name}: {best_sol.values} with fitness {best_sol.fitness}")

        best_fits.append(best_sol.fitness)

    print(f"{best_fits[0]}\n{best_fits[1]}\n{best_fits[-1]}\n mean is : {np.mean(best_fits)}\n std is : {np.std(best_fits)} ")

    all_func_names.append(name)
    all_fits_avg.append(np.mean(best_fits))
    all_fits_median.append(np.median(best_fits))
    all_fits_std.append(np.std(best_fits))

#### table

In [ ]:
df = pd.DataFrame(columns=["bench_func", "avg", "std", "median"])
df

In [ ]:
df["bench_func"] = all_func_names
df["avg"] = all_fits_avg
df["std"] = all_fits_std
df["median"] = all_fits_median

In [ ]:
df

### nD

In [ ]:
dimension = 30
pop_size = 50
crossover_rate = 0.75
mutation_rate = 0.01
max_generation = 40000
patience = 10000

results = {}
for name, [func, bounds] in unimodal_functions30D.items():
    print(f"Running GA on {name}...")
    
    best_sol, fitness_history = genetic_algorithm(
        func, bounds, dimension, pop_size, crossover_rate, mutation_rate, max_generation, patience
    )
    results[name] = {
        'best_solution': best_sol.values,
        'best_fitness': best_sol.fitness,
        'fitness_history': fitness_history
    }
    print(f"Best solution for {name}: {best_sol.values} with fitness {best_sol.fitness}")

# Plot fitness history for each function
plt.figure(figsize=(12, 8))
for name, data in results.items():
    plt.plot(data['fitness_history'][100:], label=name)

plt.xlabel('Generation')
plt.ylabel('Best Fitness')
plt.title('Genetic Algorithm Performance on Benchmark Functions (30-dimensional)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Plot fitness history for each function
plt.figure(figsize=(12, 8))
for name, data in results.items():
    plt.plot(data['fitness_history'][2000:], label=name)

plt.xlabel('Generation')
plt.ylabel('Best Fitness')
plt.title('Genetic Algorithm Performance on Benchmark Functions (30-dimensional)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
print("\nSummary Results:")
print("=" * 60)
for name, data in results.items():
    print(f"{name:20}: Best Fitness = {data['best_fitness'][0]:.6e} : \n Best Solution = {data['best_solution']}")

#### statistics

In [ ]:
results = {}
all_func_names = []
all_fits_avg = []
all_fits_median = []
all_fits_std = []


for name, [func, bounds] in unimodal_functions30D.items():
    best_fits = []
    for _ in range(20):
            
        print(f"Running GA on {name}...")
        best_sol, fitness_history = genetic_algorithm(
            func, bounds, dimension, pop_size, crossover_rate, mutation_rate, max_generation, patience
        )
        results[name] = {
            'best_solution': best_sol.values,
            'best_fitness': best_sol.fitness,
            'fitness_history': fitness_history
        }
        print(f"Best solution for {name}: {best_sol.values} with fitness {best_sol.fitness}")

        best_fits.append(best_sol.fitness)

    print(f"{best_fits[0]}\n{best_fits[1]}\n{best_fits[-1]}\n mean is : {np.mean(best_fits)}\n std is : {np.std(best_fits)} ")

    all_func_names.append(name)
    all_fits_avg.append(np.mean(best_fits))
    all_fits_median.append(np.median(best_fits))
    all_fits_std.append(np.std(best_fits))

#### table

In [ ]:
df = pd.DataFrame(columns=["bench_func", "avg", "std", "median"])

In [ ]:
df["bench_func"] = all_func_names
df["avg"] = all_fits_avg
df["std"] = all_fits_std
df["median"] = all_fits_median

In [ ]:
df

## multi

### 1d

In [ ]:
dimension = 1
pop_size = 50
crossover_rate = 0.75
mutation_rate = 0.01
max_generation = 40000
patience = 1000

results = {}
for name, [func, bounds] in multimodal_1d.items():
    print(f"Running GA on {name}...")
    best_sol, fitness_history = genetic_algorithm(
        func, bounds, dimension, pop_size, crossover_rate, mutation_rate, max_generation, patience
    )
    results[name] = {
        'best_solution': best_sol.values,
        'best_fitness': best_sol.fitness,
        'fitness_history': fitness_history
    }
    print(f"Best solution for {name}: {best_sol.values} with fitness {best_sol.fitness}")

# Plot fitness history for each function
plt.figure(figsize=(12, 8))
for name, data in results.items():
    plt.plot(data['fitness_history'], label=name)

plt.xlabel('Generation')
plt.ylabel('Best Fitness')
plt.title('Genetic Algorithm Performance on Benchmark Functions (30-dimensional)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
print("\nSummary Results:")
print("=" * 60)
for name, data in results.items():
    print(f" {name}: Best Fitness = {data['best_fitness']} ")

#### statistics

In [ ]:
results = {}
all_func_names = []
all_fits_avg = []
all_fits_median = []
all_fits_std = []


for name, [func, bounds] in multimodal_1d.items():
    best_fits = []
    for _ in range(20):
            
        print(f"Running GA on {name}...")
        best_sol, fitness_history = genetic_algorithm(
            func, bounds, dimension, pop_size, crossover_rate, mutation_rate, max_generation, patience
        )
        results[name] = {
            'best_solution': best_sol.values,
            'best_fitness': best_sol.fitness,
            'fitness_history': fitness_history
        }
        print(f"Best solution for {name}: {best_sol.values} with fitness {best_sol.fitness}")

        best_fits.append(best_sol.fitness)

    print(f"{best_fits[0]}\n{best_fits[1]}\n{best_fits[-1]}\n mean is : {np.mean(best_fits)}\n std is : {np.std(best_fits)} ")

    all_func_names.append(name)
    all_fits_avg.append(np.mean(best_fits))
    all_fits_median.append(np.median(best_fits))
    all_fits_std.append(np.std(best_fits))


#### table

In [ ]:
df = pd.DataFrame(columns=["bench_func", "avg", "std", "median"])

In [ ]:
df["bench_func"] = all_func_names
df["avg"] = all_fits_avg
df["std"] = all_fits_std
df["median"] = all_fits_median

In [ ]:
df

### 2D

#### show output

In [ ]:
dimension = 2
pop_size = 50
crossover_rate = 0.75
mutation_rate = 0.01
max_generation = 40000
patience = 1000

results = {}
for name, [func, bounds] in multimodal_2d.items():
    print(f"Running GA on {name}...")
    best_sol, fitness_history = genetic_algorithm(
        func, bounds, dimension, pop_size, crossover_rate, mutation_rate, max_generation, patience
    )
    results[name] = {
        'best_solution': best_sol.values,
        'best_fitness': best_sol.fitness,
        'fitness_history': fitness_history
    }
    print(f"Best solution for {name}: {best_sol.values} with fitness {best_sol.fitness}")

# Plot fitness history for each function
plt.figure(figsize=(12, 8))
for name, data in results.items():
    plt.plot(data['fitness_history'], label=name)

plt.xlabel('Generation')
plt.ylabel('Best Fitness')
plt.title('Genetic Algorithm Performance on Benchmark Functions (30-dimensional)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
print("\nSummary Results:")
print("=" * 60)
for name, data in results.items():
    print(f"{name:20}: Best Fitness = {data['best_fitness']:.6e}")

#### statistics

In [ ]:
dimension = 2
pop_size = 50
crossover_rate = 0.75
mutation_rate = 0.01
max_generation = 40000
patience = 10000

results = {}
all_func_names = []
all_fits_avg = []
all_fits_median = []
all_fits_std = []


for name, [func, bounds] in multimodal_2d.items():
    best_fits = []
    for _ in range(20):
            
        print(f"Running GA on {name}...")
        best_sol, fitness_history = genetic_algorithm(
            func, bounds, dimension, pop_size, crossover_rate, mutation_rate, max_generation, patience
        )
        results[name] = {
            'best_solution': best_sol.values,
            'best_fitness': best_sol.fitness,
            'fitness_history': fitness_history
        }
        print(f"Best solution for {name}: {best_sol.values} with fitness {best_sol.fitness}")

        best_fits.append(best_sol.fitness)

    print(f"{best_fits[0]}\n{best_fits[1]}\n{best_fits[-1]}\n mean is : {np.mean(best_fits)}\n std is : {np.std(best_fits)} ")

    all_func_names.append(name)
    all_fits_avg.append(np.mean(best_fits))
    all_fits_median.append(np.median(best_fits))
    all_fits_std.append(np.std(best_fits))

#### table

In [ ]:
df = pd.DataFrame(columns=["bench_func", "avg", "std", "median"])

df["bench_func"] = all_func_names
df["avg"] = all_fits_avg
df["std"] = all_fits_std
df["median"] = all_fits_median

df

### 3D

#### show output

In [ ]:
dimension = 3
pop_size = 50
crossover_rate = 0.75
mutation_rate = 0.01
max_generation = 40000
patience = 1000

results = {}
for name, [func, bounds] in multimodal_3d.items():
    print(f"Running GA on {name}...")
    best_sol, fitness_history = genetic_algorithm(
        func, bounds, dimension, pop_size, crossover_rate, mutation_rate, max_generation, patience
    )
    results[name] = {
        'best_solution': best_sol.values,
        'best_fitness': best_sol.fitness,
        'fitness_history': fitness_history
    }
    print(f"Best solution for {name}: {best_sol.values} with fitness {best_sol.fitness}")

# Plot fitness history for each function
plt.figure(figsize=(12, 8))
for name, data in results.items():
    plt.plot(data['fitness_history'], label=name)

plt.xlabel('Generation')
plt.ylabel('Best Fitness')
plt.title('Genetic Algorithm Performance on Benchmark Functions (30-dimensional)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
print("\nSummary Results:")
print("=" * 60)
for name, data in results.items():
    print(f"{name:20}: Best Fitness = {data['best_fitness']}")

#### statistics

In [ ]:
results = {}
all_func_names = []
all_fits_avg = []
all_fits_median = []
all_fits_std = []


for name, [func, bounds] in multimodal_3d.items():
    best_fits = []
    for _ in range(20):
            
        print(f"Running GA on {name}...")
        best_sol, fitness_history = genetic_algorithm(
            func, bounds, dimension, pop_size, crossover_rate, mutation_rate, max_generation, patience
        )
        results[name] = {
            'best_solution': best_sol.values,
            'best_fitness': best_sol.fitness,
            'fitness_history': fitness_history
        }
        print(f"Best solution for {name}: {best_sol.values} with fitness {best_sol.fitness}")

        best_fits.append(best_sol.fitness)

    print(f"{best_fits[0]}\n{best_fits[1]}\n{best_fits[-1]}\n mean is : {np.mean(best_fits)}\n std is : {np.std(best_fits)} ")

    all_func_names.append(name)
    all_fits_avg.append(np.mean(best_fits))
    all_fits_median.append(np.median(best_fits))
    all_fits_std.append(np.std(best_fits))

#### table

In [ ]:
df = pd.DataFrame(columns=["bench_func", "avg", "std", "median"])

df["bench_func"] = all_func_names
df["avg"] = all_fits_avg
df["std"] = all_fits_std
df["median"] = all_fits_median

df

### nD

#### show output

In [ ]:
dimension = 30
pop_size = 50
crossover_rate = 0.75
mutation_rate = 0.01
max_generation = 40000
patience = 3000

results = {}
for name, [func, bounds] in multimodal_30d.items():
    print(f"Running GA on {name}...")
    best_sol, fitness_history = genetic_algorithm(
        func, bounds, dimension, pop_size, crossover_rate, mutation_rate, max_generation, patience
    )
    results[name] = {
        'best_solution': best_sol.values,
        'best_fitness': best_sol.fitness,
        'fitness_history': fitness_history
    }
    print(f"Best solution for {name}: {best_sol.values} with fitness {best_sol.fitness}")

# Plot fitness history for each function
plt.figure(figsize=(12, 8))
for name, data in results.items():
    plt.plot(data['fitness_history'], label=name)

plt.xlabel('Generation')
plt.ylabel('Best Fitness')
plt.title('Genetic Algorithm Performance on Benchmark Functions (30-dimensional)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
print("\nSummary Results:")
print("=" * 60)
for name, data in results.items():
    print(f"{name:20}: Best Fitness = {data['best_fitness']}")

#### statistics

In [ ]:
results = {}
all_func_names = []
all_fits_avg = []
all_fits_median = []
all_fits_std = []


for name, [func, bounds] in multimodal_30d.items():
    best_fits = []
    for _ in range(20):
            
        print(f"Running GA on {name}...")
        best_sol, fitness_history = genetic_algorithm(
            func, bounds, dimension, pop_size, crossover_rate, mutation_rate, max_generation, patience
        )
        results[name] = {
            'best_solution': best_sol.values,
            'best_fitness': best_sol.fitness,
            'fitness_history': fitness_history
        }
        print(f"Best solution for {name}: {best_sol.values} with fitness {best_sol.fitness}")

        best_fits.append(best_sol.fitness)

    print(f"{best_fits[0]}\n{best_fits[1]}\n{best_fits[-1]}\n mean is : {np.mean(best_fits)}\n std is : {np.std(best_fits)} ")

    all_func_names.append(name)
    all_fits_avg.append(np.mean(best_fits))
    all_fits_median.append(np.median(best_fits))
    all_fits_std.append(np.std(best_fits))

#### table

In [ ]:
df = pd.DataFrame(columns=["bench_func", "avg", "std", "median"])

df["bench_func"] = all_func_names
df["avg"] = all_fits_avg
df["std"] = all_fits_std
df["median"] = all_fits_median

df

# PSO

In [ ]:
import numpy as np

class Particle:
    def __init__(self, bounds, dimension, func):
        self.position = np.random.uniform(bounds[0], bounds[1], dimension)
        self.velocity = np.random.uniform(-abs(bounds[1]-bounds[0]), abs(bounds[1]-bounds[0]), dimension)
        self.best_position = np.copy(self.position)
        self.fitness = func([self.position])
        self.best_fitness = self.fitness

def pso(func, bounds, dimension=30, pop_size=50, w=0.7, c1=1.5, c2=1.5, max_generation=40000, patience=1000):
    # --- Initialization
    swarm = [Particle(bounds, dimension, func) for _ in range(pop_size)]
    global_best = min(swarm, key=lambda p: p.fitness)
    best_position = np.copy(global_best.position)
    best_fitness = global_best.fitness
    
    fitness_history = [best_fitness]
    no_improve_count = 0

    # --- Main loop
    for gen in range(1, max_generation + 1):
        for particle in swarm:
            # Update velocity
            r1, r2 = np.random.rand(dimension), np.random.rand(dimension)
            cognitive = c1 * r1 * (particle.best_position - particle.position)
            social = c2 * r2 * (best_position - particle.position)
            particle.velocity = w * particle.velocity + cognitive + social

            # Update position
            particle.position += particle.velocity
            # Keep inside bounds
            particle.position = np.clip(particle.position, bounds[0], bounds[1])

            # Evaluate
            particle.fitness = func([particle.position])

            # Update personal best
            if particle.fitness[0] < particle.best_fitness:
                particle.best_fitness = particle.fitness
                particle.best_position = np.copy(particle.position)

        # Update global best
        current_best = min(swarm, key=lambda p: p.fitness)
        if current_best.fitness < best_fitness:
            best_fitness = current_best.fitness
            best_position = np.copy(current_best.position)
            no_improve_count = 0
        else:
            no_improve_count += 1

        fitness_history.append(best_fitness)

        # Early stopping
        if no_improve_count >= patience:
            print(f"Early stopping at generation {gen} (no improvement in {patience} generations)")
            break

    class BestSolution:
        def __init__(self, values, fitness):
            self.values = values
            self.fitness = fitness

    return BestSolution(best_position, best_fitness), fitness_history


# PSO results

## uni

### 2D

In [ ]:
# Parameters
dimension = 2
pop_size = 50
max_generation = 40000
patience = 3000

results = {}
for name, [func, bounds] in unimodal_functions2D.items():
    print(f"Running PSO on {name}...")
    best_sol, fitness_history = pso(
        func, bounds, dimension, pop_size, 
        w=0.74, c1=1.42, c2=1.42, 
        max_generation=max_generation, patience=patience
    )
    results[name] = {
        'best_solution': best_sol.values,
        'best_fitness': best_sol.fitness,
        'fitness_history': fitness_history
    }
    print(f"Best solution for {name}: {best_sol.values} with fitness {best_sol.fitness}")

# Plot results
plt.figure(figsize=(12, 8))
for name, data in results.items():
    plt.plot(data['fitness_history'], label=name)

plt.xlabel('Generation')
plt.ylabel('Best Fitness')
plt.title('PSO Performance on Benchmark Functions (30-dimensional)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
print("\nSummary Results:")
print("=" * 60)
for name, data in results.items():
    print(f"{name:20}: Best Fitness = {data['best_fitness']}")

#### statistics

In [ ]:
results = {}
all_func_names = []
all_fits_avg = []
all_fits_median = []
all_fits_std = []


for name, [func, bounds] in unimodal_functions2D.items():
    best_fits = []
    for _ in range(20):
            
        print(f"Running PSO on {name}...")
        best_sol, fitness_history = pso(
        func, bounds, dimension, pop_size, 
        w=0.74, c1=1.42, c2=1.42, 
        max_generation=max_generation, patience=patience
    )
        results[name] = {
            'best_solution': best_sol.values,
            'best_fitness': best_sol.fitness,
            'fitness_history': fitness_history
        }
        print(f"Best solution for {name}: {best_sol.values} with fitness {best_sol.fitness}")

        best_fits.append(best_sol.fitness)

    print(f"{best_fits[0]}\n{best_fits[1]}\n{best_fits[-1]}\n mean is : {np.mean(best_fits)}\n std is : {np.std(best_fits)} ")

    all_func_names.append(name)
    all_fits_avg.append(np.mean(best_fits))
    all_fits_median.append(np.median(best_fits))
    all_fits_std.append(np.std(best_fits))


#### table

In [ ]:
df = pd.DataFrame(columns=["bench_func", "avg", "std", "median"])

df["bench_func"] = all_func_names
df["avg"] = all_fits_avg
df["std"] = all_fits_std
df["median"] = all_fits_median

df

### nD

In [ ]:
dimension = 30
pop_size = 50
crossover_rate = 0.75
mutation_rate = 0.01
max_generation = 40000
patience = 3000

results = {}
for name, [func, bounds] in unimodal_functions30D.items():
    print(f"Running PSO on {name}...")

    best_sol, fitness_history = pso(
        func, bounds, dimension, pop_size, 
        w=0.74, c1=1.42, c2=1.42, 
        max_generation=max_generation, patience=patience
    )
    results[name] = {
        'best_solution': best_sol.values,
        'best_fitness': best_sol.fitness,
        'fitness_history': fitness_history
    }
    print(f"Best solution for {name}: {best_sol.values} with fitness {best_sol.fitness[0]:.6e}")

# Plot fitness history for each function
plt.figure(figsize=(12, 8))
for name, data in results.items():
    plt.plot(data['fitness_history'][100:], label=name)

plt.xlabel('Generation')
plt.ylabel('Best Fitness')
plt.title('Genetic Algorithm Performance on Benchmark Functions (30-dimensional)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
print("\nSummary Results:")
print("=" * 60)
for name, data in results.items():
    print(f"{name:20}: Best Fitness = {data['best_fitness']}")

#### statistics

In [ ]:
results = {}
all_func_names = []
all_fits_avg = []
all_fits_median = []
all_fits_std = []


for name, [func, bounds] in unimodal_functions30D.items():
    best_fits = []
    for _ in range(20):
            
        print(f"Running PSO on {name}...")
        best_sol, fitness_history = pso(
        func, bounds, dimension, pop_size, 
        w=0.74, c1=1.42, c2=1.42, 
        max_generation=max_generation, patience=patience
    )
        results[name] = {
            'best_solution': best_sol.values,
            'best_fitness': best_sol.fitness,
            'fitness_history': fitness_history
        }
        print(f"Best solution for {name}: {best_sol.values} with fitness {best_sol.fitness}")

        best_fits.append(best_sol.fitness)

    print(f"{best_fits[0]}\n{best_fits[1]}\n{best_fits[-1]}\n mean is : {np.mean(best_fits)}\n std is : {np.std(best_fits)} ")

    all_func_names.append(name)
    all_fits_avg.append(np.mean(best_fits))
    all_fits_median.append(np.median(best_fits))
    all_fits_std.append(np.std(best_fits))

#### table

In [ ]:
df = pd.DataFrame(columns=["bench_func", "avg", "std", "median"])

df["bench_func"] = all_func_names
df["avg"] = all_fits_avg
df["std"] = all_fits_std
df["median"] = all_fits_median

df

## multi

### 1D

In [ ]:
dimension = 1
pop_size = 50
crossover_rate = 0.75
mutation_rate = 0.01
max_generation = 40000
patience = 3000

results = {}
for name, [func, bounds] in multimodal_1d.items():
    print(f"Running PSO on {name}...")

    best_sol, fitness_history = pso(
        func, bounds, dimension, pop_size, 
        w=0.74, c1=1.42, c2=1.42, 
        max_generation=max_generation, patience=patience
    )
    results[name] = {
        'best_solution': best_sol.values,
        'best_fitness': best_sol.fitness,
        'fitness_history': fitness_history
    }
    print(f"Best solution for {name}: {best_sol.values} with fitness {best_sol.fitness[0]:.6e}")

# Plot fitness history for each function
plt.figure(figsize=(12, 8))
for name, data in results.items():
    plt.plot(data['fitness_history'][100:], label=name)

plt.xlabel('Generation')
plt.ylabel('Best Fitness')
plt.title('Genetic Algorithm Performance on Benchmark Functions (30-dimensional)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
print("\nSummary Results:")
print("=" * 60)
for name, data in results.items():
    print(f"{name:20}: Best Fitness = {data['best_fitness']}")

#### statistics

In [ ]:
results = {}
all_func_names = []
all_fits_avg = []
all_fits_median = []
all_fits_std = []


for name, [func, bounds] in multimodal_1d.items():
    best_fits = []
    for _ in range(20):
            
        print(f"Running PSO on {name}...")
        best_sol, fitness_history = pso(
        func, bounds, dimension, pop_size, 
        w=0.74, c1=1.42, c2=1.42, 
        max_generation=max_generation, patience=patience
    )
        results[name] = {
            'best_solution': best_sol.values,
            'best_fitness': best_sol.fitness,
            'fitness_history': fitness_history
        }
        print(f"Best solution for {name}: {best_sol.values} with fitness {best_sol.fitness}")

        best_fits.append(best_sol.fitness)

    print(f"{best_fits[0]}\n{best_fits[1]}\n{best_fits[-1]}\n mean is : {np.mean(best_fits)}\n std is : {np.std(best_fits)} ")

    all_func_names.append(name)
    all_fits_avg.append(np.mean(best_fits))
    all_fits_median.append(np.median(best_fits))
    all_fits_std.append(np.std(best_fits))


#### table

In [ ]:
df = pd.DataFrame(columns=["bench_func", "avg", "std", "median"])

df["bench_func"] = all_func_names
df["avg"] = all_fits_avg
df["std"] = all_fits_std
df["median"] = all_fits_median

df

### 2D

In [ ]:
dimension = 2
pop_size = 50
crossover_rate = 0.75
mutation_rate = 0.01
max_generation = 40000
patience = 3000

results = {}
for name, [func, bounds] in multimodal_2d.items():
    print(f"Running PSO on {name}...")

    best_sol, fitness_history = pso(
        func, bounds, dimension, pop_size, 
        w=0.74, c1=1.42, c2=1.42, 
        max_generation=max_generation, patience=patience
    )
    results[name] = {
        'best_solution': best_sol.values,
        'best_fitness': best_sol.fitness,
        'fitness_history': fitness_history
    }
    print(f"Best solution for {name}: {best_sol.values} with fitness {best_sol.fitness[0]:.6e}")

# Plot fitness history for each function
plt.figure(figsize=(12, 8))
for name, data in results.items():
    plt.plot(data['fitness_history'][100:], label=name)

plt.xlabel('Generation')
plt.ylabel('Best Fitness')
plt.title('Genetic Algorithm Performance on Benchmark Functions (30-dimensional)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
print("\nSummary Results:")
print("=" * 60)
for name, data in results.items():
    print(f"{name:20}: Best Fitness = {data['best_fitness']}")

#### statistics

In [ ]:
results = {}
all_func_names = []
all_fits_avg = []
all_fits_median = []
all_fits_std = []


for name, [func, bounds] in multimodal_2d.items():
    best_fits = []
    for _ in range(20):
            
        print(f"Running PSO on {name}...")
        best_sol, fitness_history = pso(
        func, bounds, dimension, pop_size, 
        w=0.74, c1=1.42, c2=1.42, 
        max_generation=max_generation, patience=patience
    )
        results[name] = {
            'best_solution': best_sol.values,
            'best_fitness': best_sol.fitness,
            'fitness_history': fitness_history
        }
        print(f"Best solution for {name}: {best_sol.values} with fitness {best_sol.fitness}")

        best_fits.append(best_sol.fitness)

    print(f"{best_fits[0]}\n{best_fits[1]}\n{best_fits[-1]}\n mean is : {np.mean(best_fits)}\n std is : {np.std(best_fits)} ")

    all_func_names.append(name)
    all_fits_avg.append(np.mean(best_fits))
    all_fits_median.append(np.median(best_fits))
    all_fits_std.append(np.std(best_fits))


#### table

In [ ]:
df = pd.DataFrame(columns=["bench_func", "avg", "std", "median"])

df["bench_func"] = all_func_names
df["avg"] = all_fits_avg
df["std"] = all_fits_std
df["median"] = all_fits_median

df

### 3D

In [ ]:
dimension = 3
pop_size = 50
crossover_rate = 0.75
mutation_rate = 0.01
max_generation = 40000
patience = 3000

results = {}
for name, [func, bounds] in multimodal_3d.items():
    print(f"Running PSO on {name}...")

    best_sol, fitness_history = pso(
        func, bounds, dimension, pop_size, 
        w=0.74, c1=1.42, c2=1.42, 
        max_generation=max_generation, patience=patience
    )
    results[name] = {
        'best_solution': best_sol.values,
        'best_fitness': best_sol.fitness,
        'fitness_history': fitness_history
    }
    print(f"Best solution for {name}: {best_sol.values} with fitness {best_sol.fitness[0]:.6e}")

# Plot fitness history for each function
plt.figure(figsize=(12, 8))
for name, data in results.items():
    plt.plot(data['fitness_history'][100:], label=name)

plt.xlabel('Generation')
plt.ylabel('Best Fitness')
plt.title('Genetic Algorithm Performance on Benchmark Functions (30-dimensional)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
print("\nSummary Results:")
print("=" * 60)
for name, data in results.items():
    print(f"{name:20}: Best Fitness = {data['best_fitness']}")

#### statistics

In [ ]:
results = {}
all_func_names = []
all_fits_avg = []
all_fits_median = []
all_fits_std = []


for name, [func, bounds] in multimodal_3d.items():
    best_fits = []
    for _ in range(20):
            
        print(f"Running PSO on {name}...")
        best_sol, fitness_history = pso(
        func, bounds, dimension, pop_size, 
        w=0.74, c1=1.42, c2=1.42, 
        max_generation=max_generation, patience=patience
    )
        results[name] = {
            'best_solution': best_sol.values,
            'best_fitness': best_sol.fitness,
            'fitness_history': fitness_history
        }
        print(f"Best solution for {name}: {best_sol.values} with fitness {best_sol.fitness}")

        best_fits.append(best_sol.fitness)

    print(f"{best_fits[0]}\n{best_fits[1]}\n{best_fits[-1]}\n mean is : {np.mean(best_fits)}\n std is : {np.std(best_fits)} ")

    all_func_names.append(name)
    all_fits_avg.append(np.mean(best_fits))
    all_fits_median.append(np.median(best_fits))
    all_fits_std.append(np.std(best_fits))


#### table

In [ ]:
df = pd.DataFrame(columns=["bench_func", "avg", "std", "median"])

df["bench_func"] = all_func_names
df["avg"] = all_fits_avg
df["std"] = all_fits_std
df["median"] = all_fits_median

df

### 30D

In [ ]:
dimension = 30
pop_size = 50
crossover_rate = 0.75
mutation_rate = 0.01
max_generation = 40000
patience = 3000

results = {}
for name, [func, bounds] in multimodal_30d.items():
    print(f"Running PSO on {name}...")

    best_sol, fitness_history = pso(
        func, bounds, dimension, pop_size, 
        w=0.74, c1=1.42, c2=1.42, 
        max_generation=max_generation, patience=patience
    )
    results[name] = {
        'best_solution': best_sol.values,
        'best_fitness': best_sol.fitness,
        'fitness_history': fitness_history
    }
    print(f"Best solution for {name}: {best_sol.values} with fitness {best_sol.fitness[0]:.6e}")

# Plot fitness history for each function
plt.figure(figsize=(12, 8))
for name, data in results.items():
    plt.plot(data['fitness_history'][100:], label=name)

plt.xlabel('Generation')
plt.ylabel('Best Fitness')
plt.title('Genetic Algorithm Performance on Benchmark Functions (30-dimensional)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
print("\nSummary Results:")
print("=" * 60)
for name, data in results.items():
    print(f"{name:20}: Best Fitness = {data['best_fitness']}")

#### statistics

In [ ]:
results = {}
all_func_names = []
all_fits_avg = []
all_fits_median = []
all_fits_std = []


for name, [func, bounds] in multimodal_30d.items():
    best_fits = []
    for _ in range(20):
            
        print(f"Running PSO on {name}...")
        best_sol, fitness_history = pso(
        func, bounds, dimension, pop_size, 
        w=0.74, c1=1.42, c2=1.42, 
        max_generation=max_generation, patience=patience
    )
        results[name] = {
            'best_solution': best_sol.values,
            'best_fitness': best_sol.fitness,
            'fitness_history': fitness_history
        }
        print(f"Best solution for {name}: {best_sol.values} with fitness {best_sol.fitness}")

        best_fits.append(best_sol.fitness)

    print(f"{best_fits[0]}\n{best_fits[1]}\n{best_fits[-1]}\n mean is : {np.mean(best_fits)}\n std is : {np.std(best_fits)} ")

    all_func_names.append(name)
    all_fits_avg.append(np.mean(best_fits))
    all_fits_median.append(np.median(best_fits))
    all_fits_std.append(np.std(best_fits))


#### table

In [ ]:
df = pd.DataFrame(columns=["bench_func", "avg", "std", "median"])

df["bench_func"] = all_func_names
df["avg"] = all_fits_avg
df["std"] = all_fits_std
df["median"] = all_fits_median

df